# Inverse problem of the reconstruction of the Neumann BCs for Laplace's equation

__Formulation of Inverse Problem__

__Direct Problem__
- Equation: $A * x = f$
- $A$: Matrix of Direct Problem
  - shape: ($\text{dof}$, $\text{dof}$)
- $x$: Solution of Direct Problem
  - shape: ($\text{dof}$)
- $f$: Load Vector of Direct Problem
  - shape: ($\text{dof}$)
- $F$: Projection Matrix from Normal Derivative in given points (Mass Matrix over Boundary Elements)
  - shape: ($\text{dof}$, $g$)

__Inverse Problem__
- Equation: $M * y = \text{grad}$
- $M$: Resulting Matrix of Inverse Problem
  - shape: ($3p$, $\text{dof}$)
- $y$: Normal Derivative in Boundary Points
- $\text{grad}$: Concatenation of Grad Vectors ($\text{grad}_x, \text{grad}_y, \text{grad}_z$)

__Additional Variables__
- $P_x, P_y, P_z$: Projection Matrices from Solution to Partial Derivative
  - shape: ($p$, $\text{dof}$)
- $M_x = P_x \times A^{-1} \times F$: Need batching with Direct Solvers (spsolve or better factorized to reduce amount of time)
  - Note: $A$ is singular, so we need to be careful with solving the system
- $M = (M_x, M_y, M_z)$: Concatenation of Inverse Problem Matrices
- $\text{dof}$: Degrees of Freedom of Direct Problem
- $p$: Number of Measuring Points
- $g$: Number of Boundary Points with Known Normal Derivative (Is it all boundary points?)

__About calculating M__

Computing $M$ consists of solving a lot of systems of linear equations:

$$A^{-1} * F = X$$

is same with solving SLAE with matrix rhs:

$$AX = F$$

Result of this multipication can be a big dense (90% density) matrix so we should solve SLAE with one or several columns form $F$.

Probable algorithm for getting $M$ (Under investigation, because it gives big error (e.g. `1e-2` instead of `1e-11`))

```python
for f in F:

  x = spsolve(A, f) # f.shape: (dof, 1)
  
  Mx[i] = P_x @ x
```

__Questions__

1. How to deal with $A$ is singular? It affects calculation of $M$.

2. How to assemble $F$, if we have rhs (source) in direct problem? (If equation is not Laplace's)

## Preprocessing

Import the required libraries

In [1]:
import numpy as np
import scipy as sp

from tqdm import tqdm

import pyquasar as pq

np.set_printoptions(precision=3, suppress=False)

Load the `.geo` file and generate the mesh for the problem using GMSH.

`refine_k` is the number of times the mesh is refined.

In [2]:
mesh = pq.Mesh.load("tetra.geo", refine_k=5)
mesh

<Mesh object summary 
	Numeration: global
	Domains: [<MeshDomain object summary
	Material: steel
	Total elements number: 32768
	Element type: Tetrahedron 4; Count: 32768
	Boundary type: neumann; Tag: 1; Element type: Triangle 3; Count: 1024.
	Boundary type: neumann; Tag: 2; Element type: Triangle 3; Count: 1024.
	Boundary type: neumann; Tag: 3; Element type: Triangle 3; Count: 1024.
	Boundary type: neumann; Tag: 4; Element type: Triangle 3; Count: 1024.
>]>

Define boundary conditions and the source term (if necessary).

In [3]:
# Dirichlet boundary conditions
def u(p, n):
  # return p[..., 0] ** 2 - p[..., 1] ** 2
  return 2 * p[..., 0] + 2 * p[..., 1] + 2 * p[..., 2]
  return 2 * p[..., 0] + 3 * p[..., 1] + 5 * p[..., 2] - 4


# Neumann boundary conditions
def flow(p, n):
  # return 2 * p[..., 0] * n[..., 0] - 2 * p[..., 1] * n[..., 1]
  return 2 * n[..., 0] + 2 * n[..., 1] + 2 * n[..., 2]
  return 2 * n[..., 0] + 3 * n[..., 1] + 5 * n[..., 2]

## Direct Problem

Assemble & solve the direct problem to obtain problem matrix, load vector and the solution `u`.

Note that mesh consists of only Neumann boundary conditions and therefore the solution `u` is need to be orthogonalized.

In [4]:
# Define the materials dictionary
materials = {
  "steel": {"neumann": flow, "steel": 0},
  "air": {"neumann": flow, "air": 0},
}

# Create a list of FemDomain objects from the mesh domains
domains = [pq.FemDomain(domain) for domain in mesh.domains]

# Create a FemProblem object with the domains
problem = pq.FemProblem(domains)

# Assemble the problem using the materials dictionary
problem.assembly(materials)

# Print the degree of freedom count
print(f"DOF: {problem.dof_count}")

problem._matrix = problem._matrix.tocsr()
problem.matrix.data[problem.matrix.indptr[-2] : problem.matrix.indptr[-1]] = 0
problem._matrix = problem._matrix.tocsc()
problem.matrix.data[problem.matrix.indptr[-2] : problem.matrix.indptr[-1]] = 0
problem._matrix[-1, -1] = 1
problem._matrix.eliminate_zeros()

# Get the kernel for Gram-Schmidt orthogonalization
kernel = problem.domains[0].kernel

# Solve the problem
sol = problem.solve(atol=1e-15)

# Perform the Gram-Schmidt orthogonalization
sol -= kernel[0] * (sol @ kernel[0]) / (kernel[0] @ kernel[0])

# Calculate the orthogonal solution
u_ort = u(mesh.domains[0].vertices, 0) - kernel[0] * (u(mesh.domains[0].vertices, 0) @ kernel[0]) / (kernel[0] @ kernel[0])

# Calculate the relative error
rel_err = np.linalg.norm(sol - u_ort) / np.linalg.norm(u_ort)

print(f"Relative error of direct problem solution: {rel_err:.2e}")

DOF: 6545
Relative error of direct problem solution: 9.91e-15


## Inverse Problem

Get points of interest

In [5]:
pts = problem.domains[0].vertices[2050:4100]
pts.shape

(2050, 3)

Project the solution gradient into the points of interest

In [6]:
proj_grad = problem.project_grad_into(pts, batch_size=128)

grad_x = proj_grad[0] @ sol
grad_y = proj_grad[1] @ sol
grad_z = proj_grad[2] @ sol

grad = np.concatenate([grad_x, grad_y, grad_z])

100%|██████████| 17/17 [00:05<00:00,  3.11it/s]


Add some noize to grad

In [7]:
np.linalg.norm(grad - 2) / np.linalg.norm(2 * np.ones_like(grad))

1.420450667753731e-14

In [8]:
noize = np.random.normal(0, 1e-11, grad.shape)
grad += noize * grad
np.mean(np.abs(noize * 100))

8.007804917810592e-10

In [9]:
np.linalg.norm(grad - 2) / np.linalg.norm(2 * np.ones_like(grad))

1.0069876834991545e-11

Get the Mass matrix over the Neumann type boundary

In [10]:
# POSSIBLE CHECK: F[:boundary_ids.max(),]^-1 @ load -> best approximation of the flow
F = problem.mass_boundary(["neumann"])

Initialize the inverse problem matrix

In [11]:
m_shape = pts.shape[0], F.shape[1]
Mx = np.zeros(m_shape)
My = np.zeros(m_shape)
Mz = np.zeros(m_shape)

Get factorization of direct problem matrix

In [12]:
sp.sparse.linalg.use_solver(useUmfpack = False)

In [13]:
direct_problem_solve = sp.sparse.linalg.factorized(problem.matrix)

Directly calculate the inverse problem matrix

In [14]:
# NOTE: This is not the best way to do it, but it is the most accurate in small problems
# pinv = np.linalg.inv(problem.matrix.toarray())
# Mx = proj_grad[0] @ pinv @ F
# My = proj_grad[1] @ pinv @ F
# Mz = proj_grad[2] @ pinv @ F

# M = np.concatenate([Mx, My, Mz])

Iteratively calculate the inverse problem matrix

In [15]:
# NOTE: This outputs bad approximations of the flow (4% instead of 1e-9%)

# batch_size = 512
# num_batches = F.shape[1] // batch_size + 1

# for i in tqdm(range(num_batches)):
#   start = i * batch_size
#   end = min((i + 1) * batch_size, F.shape[1])

#   rhs = F[:, start:end]
#   sol = lu.solve(rhs.toarray())

#   Mx[:, start:end] = (proj_grad[0] @ sol)
#   My[:, start:end] = (proj_grad[1] @ sol)
#   Mz[:, start:end] = (proj_grad[2] @ sol)

for i, f in tqdm(enumerate(F.T), total=F.shape[1]):
  sol = direct_problem_solve(f.T.toarray().ravel())

  Mx[:, i] = (proj_grad[0] @ sol).ravel()
  My[:, i] = (proj_grad[1] @ sol).ravel()
  Mz[:, i] = (proj_grad[2] @ sol).ravel()

# Mx = Mx.toarray()
# My = My.toarray()
# Mz = Mz.toarray()

M = np.concatenate([Mx, My, Mz])

100%|██████████| 2050/2050 [00:15<00:00, 136.51it/s]


Direct solution of the inverse problem

In [16]:
# res = np.linalg.pinv(M.T @ M) @ M.T @ grad
# rhs = F @ res
# rerr = np.linalg.norm(M @ res - grad) / np.linalg.norm(grad)
# print(f"Relative error of inverse exact problem solution (M @ y = grad check): {rerr:.2e}")

Iteratively calculate the inverse problem solution

In [17]:
res_flow, istop, itn, normr = sp.sparse.linalg.lsmr(M, grad, atol=1e-15, btol=1e-15)[:4]

rhs = F @ res_flow
rerr = np.linalg.norm(M @ res_flow - grad) / np.linalg.norm(grad)

print(f"Norm of the residual: {normr:.2e}")
print(f"Relative error of inverse problem solution (M @ y = grad): {rerr:.2e}")

Norm of the residual: 4.58e-03
Relative error of inverse problem solution (M @ y = grad): 2.92e-05


Test the solution

In [18]:
aerr = np.linalg.norm(rhs - problem.load_vector)
rerr = aerr / np.linalg.norm(problem.load_vector)
print(f"Absolute error (rhs must be near load_vector): {aerr:.2e}")
print(f"Relative error (rhs must be near load_vector): {rerr:.2e}")

Absolute error (rhs must be near load_vector): 2.72e-02
Relative error (rhs must be near load_vector): 1.83e-01


Test through direct problem

In [19]:
mat_dict = {
  "steel": {"steel": 0},
  "air": {"air": 0},
}

test_problem = pq.FemProblem(domains)

# Assemble the problem using the materials dictionary
test_problem.assembly(mat_dict)

# Print the degree of freedom count
print(f"DOF: {test_problem.dof_count}")

test_problem._matrix = test_problem._matrix.tocsr()
test_problem.matrix.data[test_problem.matrix.indptr[-2] : test_problem.matrix.indptr[-1]] = 0
test_problem._matrix = test_problem._matrix.tocsc()
test_problem.matrix.data[test_problem.matrix.indptr[-2] : test_problem.matrix.indptr[-1]] = 0
test_problem._matrix[-1, -1] = 1
test_problem._matrix.eliminate_zeros()

# Get the kernel for Gram-Schmidt orthogonalization
kernel = test_problem.domains[0].kernel

test_problem._load_vector += F @ res_flow

test_problem.factorize()
# Solve the problem
test_sol = test_problem.solve(atol=1e-15)

# Perform the Gram-Schmidt orthogonalization
test_sol -= kernel[0] * (test_sol @ kernel[0]) / (kernel[0] @ kernel[0])

rel_err = np.linalg.norm(test_sol - u_ort) / np.linalg.norm(u_ort)

print(f"Relative error of direct problem solution: {rel_err:.2e}")

DOF: 6545


Relative error of direct problem solution: 1.48e-02


In [20]:
test_grad_x = proj_grad[0] @ test_sol
test_grad_y = proj_grad[1] @ test_sol
test_grad_z = proj_grad[2] @ test_sol

test_grad = np.concatenate([test_grad_x, test_grad_y, test_grad_z])
np.linalg.norm(test_grad - 2) / np.linalg.norm(2 * np.ones_like(grad))

2.9228132148761266e-05